In [1]:
import numpy as np
import nltk

In [2]:
nltk.download('punkt')

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# https://drive.google.com/file/d/1E-Z1VCqAYpkJAEnY0sJXopKvxSXMB9tq/view?usp=sharing
file_data = drive.CreateFile({'id':'1E-Z1VCqAYpkJAEnY0sJXopKvxSXMB9tq'}) 
file_data.GetContentFile('intents.json')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

def stemming(word):
  return stemmer.stem(word.lower())

def bag_of_word(tokenized_sentence, words):
  # sentence = ["hello", "how", "are", "you"]
  # words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
  # bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]

  sentence_word = [stemming(word) for word in tokenized_sentence]
  bag = np.zeros(len(words), dtype=np.float32)
  for idx, word in enumerate(words):
    if word in sentence_word:
      bag[idx] = 1
  return bag



In [7]:
import json

with open("/content/intents.json",'r') as f:
  intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
  tag = intent['tag']
  tags.append(tag)

  for pattern in intent['patterns']:
    w = tokenize(pattern)
    all_words.extend(w)
    xy.append((w, tag))
  
ignore_words = ['?','.','!']
all_words = [stemming(w) for w in all_words if w not in ignore_words]

all_words = sorted(set(all_words))
tags = sorted(set(tags)) 

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

26 patterns
7 tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
54 unique stemmed words: ["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


In [8]:
print(xy)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thank', "'s", 'a', 'lot', '!'], 'thanks'), (['Which', 'items', 'do', 'you', 'have', '?'], 'items'), (['What', 'kinds', 'of', 'items', 'are', 'there', '?'], 'items'), (['What', 'do', 'you', 'sell', '?'], 'items'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Can', 'I', 'pay', 'with', 'Paypal', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['How', 'long', 'does', 'delivery', 'take', '?'], 'delivery'), (['How', 'long', 'does', 'shipping', 'take', '?'], 'delivery'), (['When', 'do', 'I', 'get', 'my', 'delivery', '?'], 'delivery'),

In [11]:
X_train = []
y_train = []

for pattern_sentence, tag in xy:
  bag = bag_of_word(pattern_sentence, all_words)
  label = tags.index(tag)

  X_train.append(bag)
  y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [15]:
X_train[3]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0.], dtype=float32)

In [13]:
y_train

array([3, 3, 3, 3, 3, 3, 2, 2, 2, 6, 6, 6, 6, 4, 4, 4, 5, 5, 5, 5, 0, 0,
       0, 1, 1, 1])

In [16]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [18]:
import random
from torch.utils.data import Dataset, DataLoader

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001

input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(X_train)
    self.x_data = X_train
    self.y_data = y_train

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(
    dataset=dataset, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=0 
    )

54 7


In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  for (words, labels) in train_loader:
    words = words.to(device)
    labels = labels.to(dtype=torch.long).to(device)

    outputs = model(words)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1) %100 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final loss : {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

Epoch [100/1000], Loss: 0.8222
Epoch [200/1000], Loss: 0.2092
Epoch [300/1000], Loss: 0.0214
Epoch [400/1000], Loss: 0.0159
Epoch [500/1000], Loss: 0.0118
Epoch [600/1000], Loss: 0.0047
Epoch [700/1000], Loss: 0.0027
Epoch [800/1000], Loss: 0.0009
Epoch [900/1000], Loss: 0.0016
Epoch [1000/1000], Loss: 0.0007
Final loss : 0.0007
training complete. file saved to data.pth


In [24]:
model.eval()

bot_name = 'Ganyu'
print("lets chat! (type 'quit' to exit) ")
while True:
  sentence = input("You : ")
  if sentence == 'quit':
    break
  
  sentence = tokenize(sentence)
  X = bag_of_word(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X).to(device)

  output = model(X)
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]
  if prob.item() > 0.55:
    for intent in intents['intents']:
      if tag == intent['tag']:
        print(f"{bot_name}: {random.choice(intent['responses'])}")
  else:
    print(f'{bot_name}: I do not understand.. :(')

lets chat! (type 'quit' to exit) 
You : hai
Ganyu: Hi there, how can I help?
You : can you help me?
Ganyu: Hi there, how can I help?
You : i need cofee
Ganyu: I do not understand.. :(
You : what can you do?
Ganyu: I do not understand.. :(
You : i hope we can do it
Ganyu: We accept most major credit cards, and Paypal
You : payment?
Ganyu: Hi there, what can I do for you?
You : pay?
Ganyu: I do not understand.. :(
You : i want to pay
Ganyu: We accept most major credit cards, and Paypal
You : but i dont have money
Ganyu: I do not understand.. :(
You : begone
Ganyu: Hi there, what can I do for you?
You : hallllo
Ganyu: Hi there, what can I do for you?
You : goodbye
Ganyu: Bye! Come back again soon.
You : quit
